In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import string
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter
from sklearn import ensemble
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('C://duh/Thinkful/Thinkful/Exercises/Medium_AggregatedData.csv')

In [ ]:
df.head()

In [ ]:
df.drop(['audioVersionDurationSec', 'codeBlock', 'codeBlockCount', 'collectionId', 'imageCount', 'isSubscriptionLocked', 'linksCount', 'postId', 'readingTime', 'responsesCreatedCount', 'socialRecommendsCount', 'tagsCount', 'publicationfacebookPageName', 'publicationfollowerCount', 'publicationpublicEmail', 'publicationdomain', 'publicationslug', 'publicationtwitterUsername', 'userId', 'userName', 'usersFollowedByCount', 'usersFollowedCount'], 1, inplace=True)
df.drop(['createdDatetime', 'latestPublishedDate', 'vote', 'scrappedDate', 'latestPublishedDatetime', 'slug', 'name', 'postCount', 'bio', 'firstPublishedDatetime', 'uniqueSlug', 'updatedDate', 'updatedDatetime'], 1, inplace=True)
df.drop(df.index[df.language != 'en'], inplace=True)
df = df.reset_index(drop=True)
authorindex = []
for author in df.author.unique():
    if df.text.where(df.author == author).nunique() >= 100:
        authorindex.append(author)
authorindex = authorindex[:10]
df = df.query('author in @authorindex')
df = df[:int(len(df)/100)]
df = df.reset_index(drop=True)
nlp = spacy.load('en')
df['doc'] = None
df['bow'] = None

del authorindex

sentences = pd.DataFrame()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
for x in range(len(df.text)):
    if x % 250 == 0:
        print("Processing row {}".format(x))
    #df.text[x] = re.sub(df.title[x], '', df.text[x])
    df.text[x] = df.text[x].lower()
    df.text[x] = re.sub('https://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('http://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('\\n', ' ', df.text[x])
    df.text[x] = re.sub('bibliography.*', '', df.text[x])
    df.text[x] = re.sub('[%s]' % re.escape(string.punctuation), '', df.text[x])
    df.text[x] = re.sub('\w*\d\w*', '', df.text[x])
    df.text[x] = re.sub('’', '', df.text[x])
    df.text[x] = re.sub('—', ' ', df.text[x])
    df.text[x] = re.sub('”', '', df.text[x])
    df.text[x] = re.sub('“', '', df.text[x])
    df.text[x] = re.sub('‘', '', df.text[x])
    df.text[x] = re.sub('€', '', df.text[x])
    df.text[x] = re.sub('…', ' ', df.text[x])
    df.text[x] = re.sub('\ \ ', ' ', df.text[x])
    df.text[x] = df.text[x].strip()
    if df.text[x] == '':
        df.drop([x], inplace=True)
        df = df.reset_index(drop=True)

In [ ]:
df.text[74]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
for x in range(len(df.text)):
    df.doc[x] = nlp(df.text[x])
    sents = [[sent, df.author[x]] for sent in df.doc[x].sents]
    sentences = sentences.append(sents, ignore_index=True)

In [ ]:
sentences.head()

In [ ]:
sentences.shape

In [ ]:
def bag_of_words(text):
    
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

def bow_features(sentences, common_words):
    
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    for i, sentence in enumerate(df['text_sentence']):
        
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        for word in words:
            df.loc[i, word] += 1
        
        if i % 1000 == 0:
            print("Processing row {}".format(i))
            
    return df

In [ ]:
for x in range(len(df.text)):
    df.bow[x] = bag_of_words(df.doc[x])

In [ ]:
df.head()

In [ ]:
common_words = set()
for x in range(len(df.text)):
    common_words.update(df.bow[x])

In [ ]:
#word_counts = bow_features(sentences, common_words)
print('A')
a = bow_features(sentences[:int(len(sentences)/10)].reset_index(drop=True), common_words)
print('B')
b = bow_features(sentences[int(len(sentences)/10):int(len(sentences)/10)+4500].reset_index(drop=True), common_words)
print('C')
c = bow_features(sentences[int(len(sentences)/10)+4500:int(len(sentences)/10)+9000].reset_index(drop=True), common_words)
print('D')
d = bow_features(sentences[int(len(sentences)/10)+9000:int(len(sentences)/10)+13500].reset_index(drop=True), common_words)
print('E')
e = bow_features(sentences[int(len(sentences)/10)+13500:int(len(sentences)/10)+18000].reset_index(drop=True), common_words)
print('F')
f = bow_features(sentences[int(len(sentences)/10)+18000:int(len(sentences)/10)+22500].reset_index(drop=True), common_words)
print('G')
g = bow_features(sentences[int(len(sentences)/10)+22500:int(len(sentences)/10)+27000].reset_index(drop=True), common_words)
print('H')
h = bow_features(sentences[int(len(sentences)/10)+27000:int(len(sentences)/10)+31500].reset_index(drop=True), common_words)
print('I')
i = bow_features(sentences[int(len(sentences)/10)+31500:int(len(sentences)/10)+36000].reset_index(drop=True), common_words)
print('J')
j = bow_features(sentences[int(len(sentences)/10)+36000:].reset_index(drop=True), common_words)

word_counts = word_counts.append([a, b, c, d, e, f, g, h, i, j], ignore_index=True)
del a, b, c, d, e, f, g, h, i, j

In [ ]:
word_counts.head()

In [ ]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0,
                                                   stratify=Y)

In [ ]:
X_norm = normalize(X)

X_pca = PCA(.95).fit_transform(X_norm)

# Calculate predicted values.
y_pred = KMeans(random_state=0).fit_predict(X_pca)

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_pred)
plt.scatter(X_pca[:, 2], X_pca[:, 3], c=y_pred)
plt.scatter(X_pca[:, 4], X_pca[:, 5], c=y_pred)
plt.scatter(X_pca[:, 6], X_pca[:, 7], c=y_pred)
plt.scatter(X_pca[:, 8], X_pca[:, 9], c=y_pred)
plt.scatter(X_pca[:, 10], X_pca[:, 11], c=y_pred)
plt.scatter(X_pca[:, 12], X_pca[:, 13], c=y_pred)
plt.scatter(X_pca[:, 14], X_pca[:, 15], c=y_pred)
plt.scatter(X_pca[:, 16], X_pca[:, 17], c=y_pred)
plt.scatter(X_pca[:, 18], X_pca[:, 19], c=y_pred)
plt.scatter(X_pca[:, 20], X_pca[:, 21], c=y_pred)
plt.scatter(X_pca[:, 22], X_pca[:, 23], c=y_pred)
plt.scatter(X_pca[:, 24], X_pca[:, 25], c=y_pred)
plt.scatter(X_pca[:, 26], X_pca[:, 27], c=y_pred)
plt.scatter(X_pca[:, 28], X_pca[:, 29], c=y_pred)
plt.scatter(X_pca[:, 30], X_pca[:, 31], c=y_pred)
plt.scatter(X_pca[:, 32], X_pca[:, 33], c=y_pred)
plt.scatter(X_pca[:, 34], X_pca[:, 35], c=y_pred)
plt.scatter(X_pca[:, 36], X_pca[:, 37], c=y_pred)
plt.scatter(X_pca[:, 38], X_pca[:, 39], c=y_pred)
plt.scatter(X_pca[:, 40], X_pca[:, 41], c=y_pred)
plt.scatter(X_pca[:, 42], X_pca[:, 43], c=y_pred)
plt.scatter(X_pca[:, 44], X_pca[:, 45], c=y_pred)
plt.scatter(X_pca[:, 46], X_pca[:, 47], c=y_pred)
plt.scatter(X_pca[:, 48], X_pca[:, 49], c=y_pred)
plt.scatter(X_pca[:, 50], X_pca[:, 51], c=y_pred)
plt.scatter(X_pca[:, 52], X_pca[:, 53], c=y_pred)
plt.scatter(X_pca[:, 54], X_pca[:, 55], c=y_pred)
plt.scatter(X_pca[:, 56], X_pca[:, 57], c=y_pred)
plt.scatter(X_pca[:, 58], X_pca[:, 59], c=y_pred)
plt.scatter(X_pca[:, 60], X_pca[:, 61], c=y_pred)
plt.scatter(X_pca[:, 62], X_pca[:, 63], c=y_pred)
plt.scatter(X_pca[:, 64], X_pca[:, 65], c=y_pred)
plt.scatter(X_pca[:, 66], X_pca[:, 67], c=y_pred)
plt.scatter(X_pca[:, 68], X_pca[:, 69], c=y_pred)
plt.scatter(X_pca[:, 70], X_pca[:, 71], c=y_pred)
plt.scatter(X_pca[:, 72], X_pca[:, 73], c=y_pred)
plt.scatter(X_pca[:, 74], X_pca[:, 75], c=y_pred)
plt.scatter(X_pca[:, 76], X_pca[:, 77], c=y_pred)
plt.scatter(X_pca[:, 78], X_pca[:, 79], c=y_pred)
plt.scatter(X_pca[:, 80], X_pca[:, 81], c=y_pred)
plt.scatter(X_pca[:, 82], X_pca[:, 83], c=y_pred)
plt.scatter(X_pca[:, 84], X_pca[:, 85], c=y_pred)
plt.scatter(X_pca[:, 86], X_pca[:, 87], c=y_pred)
plt.scatter(X_pca[:, 88], X_pca[:, 89], c=y_pred)
plt.scatter(X_pca[:, 90], X_pca[:, 91], c=y_pred)
plt.scatter(X_pca[:, 92], X_pca[:, 93], c=y_pred)
plt.scatter(X_pca[:, 94], X_pca[:, 95], c=y_pred)
plt.scatter(X_pca[:, 96], X_pca[:, 97], c=y_pred)
plt.scatter(X_pca[:, 98], X_pca[:, 99], c=y_pred)

plt.show()

In [ ]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
        
print('Training score: ', lr.score(X_train, y_train))
print('Test score: ', lr.score(X_test, y_test))

In [ ]:
svm = SVC(kernel='linear')
train = svm.fit(X_train, y_train)

print('Training score: ', svm.score(X_train, y_train))
print('Test score: ', svm.score(X_test, y_test))

In [ ]:
X_pca.shape

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5, 
                             min_df=2, 
                             stop_words='english', 
                             lowercase=True, 
                             use_idf=True,
                             norm=u'l2',
                             smooth_idf=True 
                            )

In [ ]:
sentences[int(len(sentences)/10)+36000:]